##### Copyright 2018 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Basic classification: Classify images of clothing

In [3]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report

print(tf.__version__)

2.8.0


## Import the Fashion MNIST dataset

In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


1.In this dataset, how many predictive classes are there in total?




In [5]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [6]:
len(class_names)

10

In [7]:
train_images = train_images / 255.0
test_images = test_images / 255.0

2.How many images are there in the training dataset and what is the size of the images?




In [8]:
train_images.shape

(60000, 28, 28)

3.What is the class of the 2022 figure in the training set?




In [9]:
class_names[train_labels[2022]]

'Sneaker'

4.What is the class of the 2022 image in the testing set




In [10]:
class_names[test_labels[2022]]

'Sneaker'

5.What is the range of the result after the normalization divided by 255?




In [11]:
print(np.min(train_images[0]), np.max(train_images[0]))

0.0 1.0


# Model1: NN

Create a model to have an image of 28*28 pixels as an input with 10 output classes.

The model composes of flatten, dropout 20%, dense with 128 neurons, and output layer to classify each image into 10 classes.



In [12]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

tf.random.set_seed(1234)

model = Sequential([
  layers.Flatten(),
  layers.Dropout(0.2),
  layers.Dense(128, activation='relu'),
  layers.Dense(len(class_names))
])

In [13]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
epochs = 10
history = model.fit(train_images, train_labels, epochs=epochs)

Epoch 1/10
1875/1875 [==============================] - 13s 6ms/step - loss: 0.5450 - accuracy: 0.8093
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4134 - accuracy: 0.8513
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3786 - accuracy: 0.8608
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3527 - accuracy: 0.8697
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3397 - accuracy: 0.8735
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3277 - accuracy: 0.8771
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3156 - accuracy: 0.8820
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3097 - accuracy: 0.8834
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3013 - accuracy: 0.8864
Epoch 10/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.29

In [15]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

313/313 - 1s - loss: 0.3579 - accuracy: 0.8713 - 519ms/epoch - 2ms/step

Test accuracy: 0.8712999820709229


In [16]:
#predict values from test set
y_pred = model.predict(test_images)
#convert predictions classes to one hot vectors
y_pred_classes = np.argmax(y_pred, axis=1)
#convert test observations to one hot vectors
#y_true = np.argmax(test_labels)
y_true = test_labels

print(classification_report(y_true, y_pred_classes,  target_names=class_names, digits=4))

              precision    recall  f1-score   support

 T-shirt/top     0.8085    0.8570    0.8320      1000
     Trouser     0.9877    0.9650    0.9762      1000
    Pullover     0.7404    0.8330    0.7840      1000
       Dress     0.8914    0.8950    0.8932      1000
        Coat     0.7645    0.8150    0.7890      1000
      Sandal     0.9934    0.9070    0.9482      1000
       Shirt     0.7517    0.5660    0.6458      1000
     Sneaker     0.9496    0.9040    0.9262      1000
         Bag     0.9704    0.9820    0.9761      1000
  Ankle boot     0.8691    0.9890    0.9252      1000

    accuracy                         0.8713     10000
   macro avg     0.8727    0.8713    0.8696     10000
weighted avg     0.8727    0.8713    0.8696     10000



# Model2: CNN

Modify Model1:

Before the flatten layer, add 2 layers of Conv2D with kernel(2*2) for 64 and 32 filters, respectively.



In [17]:
model = Sequential([
  layers.Conv2D(64, kernel_size=(2,2), activation='relu',input_shape=(28,28,1)),
  layers.Conv2D(32, kernel_size=(2,2), activation='relu'),
  layers.Flatten(),
  layers.Dropout(0.2),
  layers.Dense(128, activation='relu'),
  layers.Dense(10, activation='softmax')
])

In [18]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [19]:
epochs=10
history = model.fit(
  train_images, train_labels,
  epochs=epochs
)

Epoch 1/10
1875/1875 [==============================] - 148s 79ms/step - loss: 0.3754 - accuracy: 0.8654
Epoch 2/10
1875/1875 [==============================] - 148s 79ms/step - loss: 0.2367 - accuracy: 0.9126
Epoch 3/10
1875/1875 [==============================] - 148s 79ms/step - loss: 0.1760 - accuracy: 0.9345
Epoch 4/10
1875/1875 [==============================] - 147s 79ms/step - loss: 0.1285 - accuracy: 0.9522
Epoch 5/10
1875/1875 [==============================] - 148s 79ms/step - loss: 0.0895 - accuracy: 0.9669
Epoch 6/10
1875/1875 [==============================] - 147s 79ms/step - loss: 0.0650 - accuracy: 0.9758
Epoch 7/10
1875/1875 [==============================] - 148s 79ms/step - loss: 0.0479 - accuracy: 0.9822
Epoch 8/10
1875/1875 [==============================] - 150s 80ms/step - loss: 0.0374 - accuracy: 0.9861
Epoch 9/10
1875/1875 [==============================] - 148s 79ms/step - loss: 0.0297 - accuracy: 0.9891
Epoch 10/10
1875/1875 [==============================] 

In [20]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

313/313 - 5s - loss: 0.4635 - accuracy: 0.9128 - 5s/epoch - 15ms/step

Test accuracy: 0.9128000140190125


In [21]:
#predict values from test set
y_pred = model.predict(test_images)
#convert predictions classes to one hot vectors
y_pred_classes = np.argmax(y_pred, axis=1)
#convert test observations to one hot vectors
#y_true = np.argmax(test_labels)
y_true = test_labels

print(classification_report(y_true, y_pred_classes,  target_names=class_names, digits=4))

              precision    recall  f1-score   support

 T-shirt/top     0.8824    0.8100    0.8446      1000
     Trouser     0.9939    0.9770    0.9854      1000
    Pullover     0.8533    0.8960    0.8741      1000
       Dress     0.9140    0.9030    0.9085      1000
        Coat     0.8792    0.8590    0.8690      1000
      Sandal     0.9780    0.9780    0.9780      1000
       Shirt     0.7311    0.7830    0.7562      1000
     Sneaker     0.9738    0.9650    0.9694      1000
         Bag     0.9751    0.9790    0.9770      1000
  Ankle boot     0.9607    0.9780    0.9693      1000

    accuracy                         0.9128     10000
   macro avg     0.9141    0.9128    0.9131     10000
weighted avg     0.9141    0.9128    0.9131     10000



# Model3: CNN with max-pool & dropout

From those two Conv2D layers in Model2, add MaxPooling2D with pooling size (2*2) and then dropout 30%.

In [22]:
model = Sequential([
  layers.Conv2D(64, kernel_size=(2,2), activation='relu',input_shape=(28,28,1)),
  layers.MaxPooling2D(pool_size=(2,2)),
  layers.Conv2D(32, kernel_size=(2,2), activation='relu'),
  layers.MaxPooling2D(pool_size=(2,2)),
  layers.Flatten(),
  layers.Dropout(0.3),
  layers.Dense(128, activation='relu'),
  layers.Dense(10, activation='softmax')
])

In [23]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [24]:
epochs=10
history = model.fit(
  train_images, train_labels,
  epochs=epochs
)

Epoch 1/10
1875/1875 [==============================] - 56s 30ms/step - loss: 0.4758 - accuracy: 0.8257
Epoch 2/10
1875/1875 [==============================] - 55s 29ms/step - loss: 0.3389 - accuracy: 0.8751
Epoch 3/10
1875/1875 [==============================] - 55s 29ms/step - loss: 0.2960 - accuracy: 0.8910
Epoch 4/10
1875/1875 [==============================] - 55s 29ms/step - loss: 0.2690 - accuracy: 0.8991
Epoch 5/10
1875/1875 [==============================] - 55s 29ms/step - loss: 0.2492 - accuracy: 0.9073
Epoch 6/10
1875/1875 [==============================] - 55s 29ms/step - loss: 0.2316 - accuracy: 0.9131
Epoch 7/10
1875/1875 [==============================] - 54s 29ms/step - loss: 0.2182 - accuracy: 0.9174
Epoch 8/10
1875/1875 [==============================] - 55s 29ms/step - loss: 0.2071 - accuracy: 0.9218
Epoch 9/10
1875/1875 [==============================] - 55s 29ms/step - loss: 0.1967 - accuracy: 0.9260
Epoch 10/10
1875/1875 [==============================] - 55s 29m

In [25]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

313/313 - 2s - loss: 0.2462 - accuracy: 0.9104 - 2s/epoch - 8ms/step

Test accuracy: 0.9103999733924866


In [26]:
#predict values from test set
y_pred = model.predict(test_images)
#convert predictions classes to one hot vectors
y_pred_classes = np.argmax(y_pred, axis=1)
#convert test observations to one hot vectors
#y_true = np.argmax(test_labels)
y_true = test_labels

print(classification_report(y_true, y_pred_classes,  target_names=class_names, digits=4))

              precision    recall  f1-score   support

 T-shirt/top     0.8632    0.8710    0.8671      1000
     Trouser     0.9949    0.9810    0.9879      1000
    Pullover     0.8042    0.9160    0.8565      1000
       Dress     0.9139    0.9240    0.9189      1000
        Coat     0.8417    0.8510    0.8463      1000
      Sandal     0.9938    0.9680    0.9807      1000
       Shirt     0.8007    0.6670    0.7278      1000
     Sneaker     0.9618    0.9580    0.9599      1000
         Bag     0.9812    0.9900    0.9856      1000
  Ankle boot     0.9477    0.9780    0.9626      1000

    accuracy                         0.9104     10000
   macro avg     0.9103    0.9104    0.9093     10000
weighted avg     0.9103    0.9104    0.9093     10000

